# Libraries requirement for CoNIC environment
conda create --name CoNIC python=3.7

#essential libraries:
pip3 docopt numpy pandas tqdm scipy sklearn
#pip3 scikit-image matplotlib

for running in jupyter notebook: 
#pip3 jupyter

In [127]:
from docopt import docopt
import numpy as np
import os
import pandas as pd
from tqdm.auto import tqdm

from metrics.stats_utils import get_pq, get_multi_pq_info, get_multi_r2

#### Data should be .npy format not .tif format

In [128]:
# https://github.com/TissueImageAnalytics/CoNIC
# To get the stats for segmentation and classification, run:

# python compute_stats.py --mode="seg_class" --pred=<path_to_results> --true=<path_to_ground_truth>
# To get the stats for cellular composition prediction, run:

# python compute_stats.py --mode="regression" --pred=<path_to_results> --true=<path_to_ground_truth>

mode = ["--mode"]
pred_array = np.load('../CoNIC-main/Predictions/labels.npy')
true_array = np.load('../CoNIC-main/Ground_Truth/labels.npy')
#seg_metrics_names = ["pq", "multi_pq+"]
seg_metrics_names = ["pq", "multi_pq+"]
reg_metrics_names = ["r2"]

In [129]:
# Select only images n to m
pred_array = pred_array[4000:4100,:,:,:]
true_array = true_array[4000:4100,:,:,:]

In [130]:
nr_patches = pred_array.shape[0]

In [131]:
all_metrics = {}
pq_list = []
mpq_info_list = []

In [132]:
for patch_idx in tqdm(range(nr_patches)):
    # get a single patch
    pred = pred_array[patch_idx]
    true = true_array[patch_idx]
    # instance segmentation map
    pred_inst = pred[..., 0]
    true_inst = true[..., 0]
    # classification map
    pred_class = pred[..., 1]
    true_class = true[..., 1]
    
    for idx, metric in enumerate(seg_metrics_names):
        if metric == "pq":
            # get binary panoptic quality
            pq = get_pq(true_inst, pred_inst)
            pq = pq[0][2]
            pq_list.append(pq)
        elif metric == "multi_pq+":
            # get the multiclass pq stats info from single image
            mpq_info_single = get_multi_pq_info(true, pred)
            mpq_info = []
            # aggregate the stat info per class
            for single_class_pq in mpq_info_single:
                tp = single_class_pq[0]
                fp = single_class_pq[1]
                fn = single_class_pq[2]
                sum_iou = single_class_pq[3]
                mpq_info.append([tp, fp, fn, sum_iou])
            mpq_info_list.append(mpq_info)
        else:
            raise ValueError("%s is not supported!" % metric)

  0%|          | 0/100 [00:00<?, ?it/s]

In [133]:
pq_metrics = np.array(pq_list)
pq_metrics_avg = np.mean(pq_metrics, axis=-1)  # average over all images
if "multi_pq+" in seg_metrics_names:
    mpq_info_metrics = np.array(mpq_info_list, dtype="float")
    # sum over all the images
    total_mpq_info_metrics = np.sum(mpq_info_metrics, axis=0)

In [134]:
for idx, metric in enumerate(seg_metrics_names):
    if metric == "multi_pq+":
        mpq_list = []
        # for each class, get the multiclass PQ
        for cat_idx in range(total_mpq_info_metrics.shape[0]):
            total_tp = total_mpq_info_metrics[cat_idx][0]
            total_fp = total_mpq_info_metrics[cat_idx][1]
            total_fn = total_mpq_info_metrics[cat_idx][2]
            total_sum_iou = total_mpq_info_metrics[cat_idx][3]

            # get the F1-score i.e DQ
            dq = total_tp / (
                (total_tp + 0.5 * total_fp + 0.5 * total_fn) + 1.0e-6
            )
            # get the SQ, when not paired, it has 0 IoU so does not impact
            sq = total_sum_iou / (total_tp + 1.0e-6)
            mpq_list.append(dq * sq)
        mpq_metrics = np.array(mpq_list)
        all_metrics[metric] = [np.mean(mpq_metrics)]
    else:
        all_metrics[metric] = [pq_metrics_avg]

In [135]:
df = pd.DataFrame(all_metrics)
df = df.to_string(index=False)
print(df)

 pq  multi_pq+
1.0        1.0


## Predicting cellular composition: multi-class coefficient of determination (R2)

#### Data should be .csv format

In [154]:
pred_csv = pd.read_csv('../CoNIC-main/Predictions_count/counts.csv')
true_csv = pd.read_csv('../CoNIC-main/GT_count/counts.csv')

In [155]:
# Select only rows n to m
pred_csv = pred_csv.iloc[100:1000]
true_csv = true_csv.iloc[100:1000]

In [156]:
for idx, metric in enumerate(reg_metrics_names):
    if metric == "r2":
        # calculate multiclass coefficient of determination
        r2 = get_multi_r2(true_csv, pred_csv)
        all_metrics["multi_r2"] = [r2]
    else:
        raise ValueError("%s is not supported!" % metric)

In [157]:
df = pd.DataFrame(all_metrics)
df = df.to_string(index=False)
print(df)

 pq  multi_pq+  multi_r2
1.0        1.0       1.0
